In [50]:
import sys, os, time

import open3d as o3d
import trimesh
import openmesh as om

import cv2

import scipy as sp
from scipy.sparse.linalg import eigs
from scipy.sparse import csr_matrix, identity
from scipy.sparse.linalg import spsolve
from scipy.spatial import Delaunay, Voronoi

import numpy as np

import matplotlib.pyplot as plt

from tqdm import tqdm

%matplotlib inline

In [27]:
def read_mesh_om(path_in_models):
    return om.read_trimesh(os.path.join('..', 'models', path_in_models))
def write_mesh_om(mesh, path_in_models):
    om.write_mesh(os.path.join('..', 'models', path_in_models), mesh, vertex_color=True)
def show_mesh_o3d(plys):
    o3d.visualization.draw_geometries(plys)
def write_mesh_o3d(path, mesh):
    o3d.io.write_triangle_mesh(path, mesh)
def read_mesh_o3d(mesh_fp):
    return o3d.io.read_triangle_mesh(mesh_fp)
def read_mesh_trimesh(path_in_models):
    return trimesh.load(os.path.join('..', 'models', path_in_models))

In [28]:
help(om.write_mesh)

Help on built-in function write_mesh in module openmesh:

write_mesh(...) method of builtins.PyCapsule instance
    write_mesh(*args, **kwargs)
    Overloaded function.
    
    1. write_mesh(filename: str, mesh: openmesh.TriMesh, binary: bool = False, msb: bool = False, lsb: bool = False, swap: bool = False, vertex_normal: bool = False, vertex_color: bool = False, vertex_tex_coord: bool = False, halfedge_tex_coord: bool = False, edge_color: bool = False, face_normal: bool = False, face_color: bool = False, color_alpha: bool = False, color_float: bool = False, status: bool = False, texture_file: str = '', material_file_extension: str = '.mat') -> None
    
    2. write_mesh(filename: str, mesh: openmesh.PolyMesh, binary: bool = False, msb: bool = False, lsb: bool = False, swap: bool = False, vertex_normal: bool = False, vertex_color: bool = False, vertex_tex_coord: bool = False, halfedge_tex_coord: bool = False, edge_color: bool = False, face_normal: bool = False, face_color: bool = Fa

In [29]:
mesh = read_mesh_om('cube.obj')

In [30]:
# write_mesh_om(mesh, 'test/cube-copy.obj')

In [31]:
mesh.has_vertex_colors()

False

In [32]:
print(mesh.vertices())
set = False
for v_it in mesh.vertices():
    if not set:
        mesh.set_point(v_it, np.array([-1, -1 , 1]))
        set = True
    print(mesh.point(v_it), type(mesh.point(v_it)))
    mesh.set_color(v_it, np.array([255.,182.,193.,255.])/255.)

# for f_it in mesh.faces():
#     mesh.set_color(f_it, np.array([255.,182.,193.,255.])/255.)
# for c_it in mesh.face_colors():
#     print(c_it, type(c_it))
write_mesh_om(mesh, 'test/cube-copy.off')

[-1. -1.  1.] <class 'numpy.ndarray'>
[ 0.5 -0.5  0.5] <class 'numpy.ndarray'>
[-0.5  0.5  0.5] <class 'numpy.ndarray'>
[0.5 0.5 0.5] <class 'numpy.ndarray'>
[-0.5  0.5 -0.5] <class 'numpy.ndarray'>
[ 0.5  0.5 -0.5] <class 'numpy.ndarray'>
[-0.5 -0.5 -0.5] <class 'numpy.ndarray'>
[ 0.5 -0.5 -0.5] <class 'numpy.ndarray'>


In [94]:
INFINITY = 1e9

def get_vertices_from_trimesh(mesh):
    return np.asarray(mesh.vertices)

def get_vertice_normals_from_trimesh(mesh):
    return np.asarray(mesh.vertex_normals)

def calculate_voronoi_poles(mesh):
    
    vertices = get_vertices_from_trimesh(mesh)
    vertice_normals = get_vertice_normals_from_trimesh(mesh)

    print("Calculating Voronoi Diagram.")
    vor = Voronoi(vertices)
    vor_centers = vor.vertices
    cells = vor.regions
    cell_indices = vor.point_region

    vertices_total = vertices.shape[0]
    voronoi_poles = np.zeros(vertices.shape)
    print("Calculating the Voronoi Pole for each vertex.")
    for vi in tqdm(range(vertices_total)):
        vor_cell = cells[cell_indices[vi]]
        vertice = vertices[vi]
        vertice_normal = vertice_normals[vi]
        max_neg_proj = INFINITY
        voronoi_pole = None
        for vci in vor_cell:
            vor_center = vor_centers[vci]
            if vci == -1:
                continue
            proj = np.dot(vor_center - vertice, vertice_normal)
            if proj < max_neg_proj:
                max_neg_proj = proj
                voronoi_pole = vor_center
        voronoi_poles[vi] = voronoi_pole

    return voronoi_poles

In [96]:
mesh = read_mesh_trimesh('armadillo.obj')
voronoi_poles = calculate_voronoi_poles(mesh)
# print(voronoi_poles)

Calculating Voronoi Diagram.
Calculating the Voronoi Pole for each vertex.


100%|██████████| 25193/25193 [00:00<00:00, 27775.73it/s]


In [ ]:
#         v0 ---------- v2
#        /        +     /
#      /      vn       /
#    /     +          /   
#  v1 ---------------v3
def reduce_Angles(mesh):

    faces = np.asarray(mesh.faces)
    edges_unique = np.asarray(mesh.edges_unique)
    face_adjacency = np.asarray(mesh.face_adjacency)
    face_angles = np.asarray(mesh.face_angles)
    vertices = np.asarray(mesh.vertices)

    f_is_del = np.zeros((len(mesh.faces)),dtype=bool) # f is deleted or not
    f_to_add = np.asarray(mesh.faces) # all v indexs. Note that here it only adds points and dont remove any.
    v_to_add = np.asarray(mesh.vertices) # positions
    
    v_count = mesh.vertices.shape[0]
    face_count = faces.shape[0]
    
    for face_index in range(face_count):
        face = faces[face_index]
        if not f_is_del[face_index] and max(face_angles[face_index]) > (2./3.)*np.pi: # 2.1
            
            # get v0,v1,v2
            v0 = face[np.argmax(face_angles[face_index])] # obtuse_vertex
            v1, v2 = face[face!=v0] # v1,v2 are indexes.                

            # add vertex vn
            v_count += 1
            
            v_new = cal_projection(*vertices[[v0, v1, v2]])
            v_to_add = np.vstack((v_to_add, v_new))

            # get indexs of faces to be deleted
            # shared_faces_index = get_shared_face_index(mesh, v1, v2)
            shared_faces_index = get_adj_face(edges_unique, face_adjacency, v1, v2)
            f_is_del[shared_faces_index]= True

            # # get v3 from the faces
            v_vertices = np.asarray(np.unique(faces[shared_faces_index]))
            #print((v0,v1,v2))
            #print(v_vertices)
            #print(np.setdiff1d(v_vertices, (v0,v1,v2)))
            v3 = int(np.setdiff1d(v_vertices, (v0,v1,v2))) # ----may cause problems----                

            # # add faces
            vn = v_count - 1
            f_to_add = np.vstack((f_to_add, [v0,v1,vn]))
            f_to_add = np.vstack((f_to_add, [v0,vn,v2])) # the order matters! if go v0v2vn, not work
            f_to_add = np.vstack((f_to_add, [vn,v1,v3])) # why
            f_to_add = np.vstack((f_to_add, [v2,vn,v3]))                
            f_is_del = np.hstack((f_is_del, [False,False,False,False]))

    faces_new = f_to_add[~f_is_del]
    print(f_to_add.shape[0], faces_new.shape[0])
    #print(np.where(f_is_del))
    v_add = v_count - mesh.vertices.shape[0]
    mesh.vertices = v_to_add
    print(mesh.vertices.shape, v_to_add.shape)
    mesh.faces = faces_new

    return v_add
    